## 下载数据集

In [1]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset


ds = load_dataset("thu-coai/lccc","large",cache_dir='./data')

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['dialog'],
        num_rows: 12007759
    })
})

In [4]:
ds['train'][0]

{'dialog': ['火锅 我 在 重庆 成都 吃 了 七八 顿 火锅',
  '哈哈哈哈 ！ 那 我 的 嘴巴 可能 要 烂掉 ！',
  '不会 的 就是 好 油腻']}

In [6]:
from datasets import load_dataset

def process_dialog(example):
    # 去掉每句话首尾空格，并用单空格连接词，保证句子整洁
    processed = [''.join(sentence.split()) for sentence in example['dialog']]
    # 只保留对话轮数≥10的样本
    if len(processed) >= 10:
        return {'dialog': processed}
    else:
        return {'dialog': []}  # 空列表，后面过滤掉

dataset = load_dataset("thu-coai/lccc","large",cache_dir='./data')

processed_datasets = {}
for split in dataset.keys():
    print(f"\n处理 {split} 数据集...")
    processed = dataset[split].map(process_dialog)
    processed = processed.filter(lambda x: len(x['dialog']) > 0)
    processed_datasets[split] = processed
    print(f"原始对话数量: {len(dataset[split])}")
    print(f"处理后对话数量: {len(processed)}")
    print("前3个示例：")
    for d in processed['dialog'][:3]:
        print(d)
        print()

# 保存数据集
for split, ds in processed_datasets.items():
    ds.save_to_disk(f"lccc-clean-{split}")


处理 train 数据集...


Map:   0%|          | 0/12007759 [00:00<?, ? examples/s]

Filter:   0%|          | 0/12007759 [00:00<?, ? examples/s]

原始对话数量: 12007759
处理后对话数量: 66222
前3个示例：
['你自卑吗', '我不是颜控我是钱控', '钱控说明什么', '说明你欠我钱', '有欠条吗', '木有欠条就不还了嘛', '没欠条也算欠钱吗', '给你两分钟赶快写欠条，不然我就哭给你看', '有本事你哭', '有本事你哄我', '没本事！哭就哭！', '那你先哭。。。', '我为什么要哭', '你不哭，我怎么知道怎么哭', '我也不知道怎么哭', '你个来，我打两下就好了']

['我看到了你的照片了', '说说你判断的依据是什么', '长得丑', ':-)谢谢', '我说难听的话，你还谢谢我。', '...呵呵我就知道你会这么说', '睡觉了？', '睡着了就什么也不想了', '不理我，那我就睡觉了', '你看大家都没发话呢', '我累了，明天还要上班，', '看出来了，你眼袋都快拖地了，赶紧睡觉吧']

['谢谢关心', '这么客气', '那该怎么说……', '你带情绪…', '换你能不带情绪吗，除非不是真感情', '哎～～', '没什么拉，', '别太计较，会很累，也很不开心，有时候朋友比恋人好', '好啦，知道拉，计较的话又不是这样了，', '嗯嗯，开心起来啦', '还可以吧', '嗯，']



Saving the dataset (0/1 shards):   0%|          | 0/66222 [00:00<?, ? examples/s]

In [7]:
from datasets import load_from_disk

processed_dataset = load_from_disk("./lccc-clean-train")  # 路径改成你保存的路径

print(f"处理后对话数量: {len(processed_dataset)}")
print("前3个示例：")
for i in range(3):
    print(processed_dataset[i]['dialog'])
    print()

处理后对话数量: 66222
前3个示例：
['你自卑吗', '我不是颜控我是钱控', '钱控说明什么', '说明你欠我钱', '有欠条吗', '木有欠条就不还了嘛', '没欠条也算欠钱吗', '给你两分钟赶快写欠条，不然我就哭给你看', '有本事你哭', '有本事你哄我', '没本事！哭就哭！', '那你先哭。。。', '我为什么要哭', '你不哭，我怎么知道怎么哭', '我也不知道怎么哭', '你个来，我打两下就好了']

['我看到了你的照片了', '说说你判断的依据是什么', '长得丑', ':-)谢谢', '我说难听的话，你还谢谢我。', '...呵呵我就知道你会这么说', '睡觉了？', '睡着了就什么也不想了', '不理我，那我就睡觉了', '你看大家都没发话呢', '我累了，明天还要上班，', '看出来了，你眼袋都快拖地了，赶紧睡觉吧']

['谢谢关心', '这么客气', '那该怎么说……', '你带情绪…', '换你能不带情绪吗，除非不是真感情', '哎～～', '没什么拉，', '别太计较，会很累，也很不开心，有时候朋友比恋人好', '好啦，知道拉，计较的话又不是这样了，', '嗯嗯，开心起来啦', '还可以吧', '嗯，']



In [8]:
processed_dataset

Dataset({
    features: ['dialog'],
    num_rows: 66222
})

## 注意事项